In [3]:
import pandas as pd

df_tennis = pd.DataFrame.from_csv('id3.csv')

print("\n Given Play Tennis Data Set:\n\n", df_tennis)

df_tennis.keys()[0]

def entropy(probs):
    import math
    return sum( [-prob*math.log(prob, 2) for prob in probs] )

def entropy_of_list(a_list):
    from collections import Counter
    cnt = Counter(x for x in a_list)
    num_instances = len(a_list)*1.0
    print("\n Number of Instances of the Current Sub Class is {0}:".format(num_instances ))
    probs = [x / num_instances for x in cnt.values()]
    print("\n Classes:",min(cnt),max(cnt))
    print(" \n Probabilities of Class {0} is {1}:".format(min(cnt),min(probs)))
    print(" \n Probabilities of Class {0} is {1}:".format(max(cnt),max(probs)))
    return entropy(probs)

print("\n INPUT DATA SET FOR ENTROPY CALCULATION:\n", df_tennis['PlayTennis'])

total_entropy = entropy_of_list(df_tennis['PlayTennis'])

print("\n Total Entropy of PlayTennis Data Set: ",total_entropy)

def information_gain(df, split_attribute_name, target_attribute_name, trace=0):
    print("\nInformation Gain Calculation of ",split_attribute_name)
    df_split = df.groupby(split_attribute_name)
    nobs = len(df.index) * 1.0
    df_agg_ent = df_split.agg({target_attribute_name : [entropy_of_list, lambda x:len(x)/nobs] })[target_attribute_name]
    df_agg_ent.columns = ['Entropy', 'PropObservations']
    new_entropy = sum( df_agg_ent['Entropy'] * df_agg_ent['PropObservations'] )
    old_entropy = entropy_of_list(df[target_attribute_name])
    return old_entropy - new_entropy


def id3(df, target_attribute_name, attribute_names, default_class=None):

    from collections import Counter
    cnt = Counter(x for x in df[target_attribute_name])
    
    if len(cnt) == 1:
        return next(iter(cnt))
    
    elif df.empty or (not attribute_names):
        return default_class
    
    else:
        default_class = max(cnt.keys())
        gainz = [information_gain(df, attr, target_attribute_name) for attr in attribute_names]
        index_of_max = gainz.index(max(gainz))
        best_attr = attribute_names[index_of_max]
        tree = {best_attr:{}}
        remaining_attribute_names = [i for i in attribute_names if i != best_attr]
        for attr_val, data_subset in df.groupby(best_attr):
            subtree = id3(data_subset,target_attribute_name,remaining_attribute_names,default_class)
            tree[best_attr][attr_val] = subtree
        return tree

attribute_names = list(df_tennis.columns)
print("\nList of Attributes:", attribute_names)
attribute_names.remove('PlayTennis')

print("\nPredicting Attributes:", attribute_names)

from pprint import pprint

tree = id3(df_tennis,'PlayTennis',attribute_names)
print("\n\nThe Resultant Decision Tree is :\n")
pprint(tree)
attribute = next(iter(tree))
print("Best Attribute :\n",attribute)
print("Tree Keys:\n",tree[attribute].keys())


 Given Play Tennis Data Set:

          Temperature Humidity   Windy PlayTennis
Outlook                                         
Sunny            Hot     High    Weak         No
Sunny            Hot     High  Strong         No
Overcast         Hot     High    Weak        Yes
Rainy           Mild     High    Weak        Yes
Rainy           Cool   Normal    Weak        Yes
Rainy           Cool   Normal  Strong         No
Overcast        Cool   Normal  Strong        Yes
Sunny           Mild     High    Weak         No
Sunny           Cool   Normal    Weak        Yes
Rainy           Mild   Normal    Weak        Yes
Sunny           Mild   Normal  Strong        Yes
Overcast        Mild     High  Strong        Yes
Overcast         Hot   Normal    Weak        Yes
Rainy           Mild     High  Strong         No

 INPUT DATA SET FOR ENTROPY CALCULATION:
 Outlook
Sunny        No
Sunny        No
Overcast    Yes
Rainy       Yes
Rainy       Yes
Rainy        No
Overcast    Yes
Sunny        No
Sunny

/home/udipta/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  This is separate from the ipykernel package so we can avoid doing imports until
